In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib as mp

# Import csv

## Main import

In [35]:
with open('../data/ESP_PUBLIC.IDENTITE_ARBRE.csv', newline='', encoding='utf-8-sig') as csvfile:
    # use "," separator to generate column value as list element
    # cs.reader return a class object of "_csv.reader"
    dataTemp = csv.reader(csvfile, delimiter=',')
    
    # Use list() to convert the csv object into a list, of list, depending on how the object is built
    # Hard to understand how list(csv_object) behaves 
    data = list(dataTemp)

## Copy the dataset to advoid re-import

In [36]:
dataset = data

## Utils

In [96]:
def get_header(data:list) -> list:
    if len(data) > 0:
        return data[0]
    else:
        raise ValueError("The dataset sent is empty...")

In [97]:
def get_dict_colName_colNum(data:list) -> dict:
    dictTemp = {}
    
    for el in get_header(data):
        dictTemp[el] = header.index(el)
        
    return dictTemp
print(get_dict_colName_colNum(dataset))

{'ELEM_POINT_ID': 0, 'CODE': 1, 'NOM': 2, 'GENRE': 3, 'GENRE_DESC': 4, 'CATEGORIE': 5, 'CATEGORIE_DESC': 6, 'SOUS_CATEGORIE': 7, 'SOUS_CATEGORIE_DESC': 8, 'CODE_PARENT': 9, 'CODE_PARENT_DESC': 10, 'ADR_SECTEUR': 11, 'BIEN_REFERENCE': 12, 'GENRE_BOTA': 13, 'ESPECE': 14, 'VARIETE': 15, 'STADEDEDEVELOPPEMENT': 16, 'EQUIPE': 17, 'REMARQUES': 18, 'ANNEEDEPLANTATION': 19, 'RAISONDEPLANTATION': 20, 'TRAITEMENTCHENILLES': 21, 'COURRIER': 22, 'IDENTIFIANTPLU': 23, 'TYPEIMPLANTATIONPLU': 24, 'INTITULEPROTECTIONPLU': 25, 'ANNEEABATTAGE': 26, 'ESSOUCHEMENT': 27, 'DIAMETREARBRE': 28, 'CAUSEABATTAGE': 29, 'COLLECTIVITE': 30, 'GeoJSON': 31}


In [98]:
def is_in_header(colName:str, data:list) -> bool:
    return colName in get_header(data)
print(is_in_header("GENRE_DESC", dataset))

True


In [131]:
def get_col_value_from_name(colName:str, data:list) -> str:
    
    if is_in_header(colName, data):
        return get_dict_colName_colNum(data)[colName]
    
# print(get_col_value_from_name(colName="CODE_PARENT", data=dataset))

9


In [136]:
def get_col_values(colName:str, data:list) -> list:
    dataCopy = data.copy()
    col = [el[get_col_value_from_name(colName, dataCopy)] for el in dataCopy]
    
    return col
# print(get_col_values("ANNEEDEPLANTATION", dataset))

# Main computation

## Afficher deux premieres lignes

In [137]:
# Slice and extract the first two list ie the first two lignes
header = get_header(dataset)
lignes2 = dataset[0:2]

specDataset = {
    "rows" : len(dataset),
    "columns" : len(dataset[0])
}

## Afficher colonne n°2

In [138]:
# We see that to access the row is [i] after lignes2. 
# Then we have the list corresponding to the row i-1 (start from 0)
# In this list, we use [i] to acces the i element corresponding to the column
ligne1_col2 = lignes2[0][1]
ligne2_col2 = lignes2[1][1]

In [139]:
print(ligne1_col2 + " : " + ligne2_col2)

CODE : ESP29285


## Infos année plantation

In [143]:
# print(get_header(dataset))

In [144]:
anneePlantation = get_col_values("ANNEEDEPLANTATION", dataset)
anneePlantation

['ANNEEDEPLANTATION',
 '1977',
 '2001',
 '1975',
 '1978',
 '1960',
 '1960',
 '2000',
 '1944',
 '1944',
 '1991',
 '2012',
 '1980',
 '1992',
 '1982',
 '1986',
 '1977',
 '2004',
 '2004',
 '1965',
 '1965',
 '1991',
 '1960',
 '1924',
 '2004',
 '1960',
 '1960',
 '1982',
 '1990',
 '2010',
 '1999',
 '1998',
 '1975',
 '1975',
 '2001',
 '2001',
 '2001',
 '1965',
 '1965',
 '1979',
 '1960',
 '1978',
 '1978',
 '2006',
 '1975',
 '1975',
 '2001',
 '2001',
 '1994',
 '1995',
 '1995',
 '1995',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1980',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '2006',
 '1976',
 '1976',
 '1985',
 '1975',
 '1975',
 '1976',
 '1976',
 '1972',
 '1980',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1975',
 '1980',
 '1975',
 '1975',
 '1995',
 '2009',
 '1994',
 '2004',
 '2011',
 '2010',
 '1980',
 '2009',
 '1980',
 '2009

In [148]:
def get_empty_value(col:list):
    
    for el in col:
        if not el:
            print("--> empty <--")
        else:
            print(el)

In [149]:
get_empty_value(anneePlantation)

ANNEEDEPLANTATION
1977
2001
1975
1978
1960
1960
2000
1944
1944
1991
2012
1980
1992
1982
1986
1977
2004
2004
1965
1965
1991
1960
1924
2004
1960
1960
1982
1990
2010
1999
1998
1975
1975
2001
2001
2001
1965
1965
1979
1960
1978
1978
2006
1975
1975
2001
2001
1994
1995
1995
1995
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1980
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
2006
1976
1976
1985
1975
1975
1976
1976
1972
1980
1975
1975
1975
1975
1975
1975
1975
1975
1975
1980
1975
1975
1995
2009
1994
2004
2011
2010
1980
2009
1980
2009
1950
2005
2005
2005
1950
1950
1950
1950
2006
2006
1965
1965
1978
1978
1971
1975
2001
2001
1994
2001
1975
1975
1984
1975
1987
1975
1980
2006
1975
1975
1965
1975
1980
1983
1982
1976
2011
1970
2009
1981
1981
1996
1983
1983
1983
1985
1955
1955
1987
1987
2003
1978
1995
1995
2010
2010
1979
1992
1999
2003
2002
1987
1975
1975
2001
1996
1995
1999
1970
1970
1999
2000
1999
1970
1970
1992
1992
2007
1972
1975
1975
1975
1976
1976
1979
1972
1972
19